In [1]:
import pandas as pd
import requests
import io
import os
import time
import urllib.parse

def download_and_verify_proteomes(input_csv, data_output_csv, metadata_output_csv):
    """
    Steps 2-5: Reads processed proteome list, downloads sequences from UniProt,
    verifies counts, and saves data to CSV and metadata to a separate CSV.
    """
    
    # --- Step 2: Read the proteome id and organism id from input_file ---
    print(f"Reading input file: {input_csv}...")
    try:
        df = pd.read_csv(input_csv)
    except FileNotFoundError:
        print(f"Error: The file '{input_csv}' was not found.")
        return

    # Check for necessary columns
    required_columns = ['Proteome Id', 'Organism Id', 'Protein count']
    if not all(col in df.columns for col in required_columns):
        print(f"Error: Input file must contain columns: {required_columns}")
        return

    # Initialize Metadata List
    metadata_records = []

    # Initialize Output CSV (Write Header if file doesn't exist)
    # We include 'Entry' (Accession) as it comes with the TSV download and is useful
    header = ["Proteome Id", "Organism Id", "Entry", "Sequence"]
    
    if not os.path.exists(data_output_csv):
        pd.DataFrame(columns=header).to_csv(data_output_csv, index=False)
        print(f"Created new data file: {data_output_csv}")
    else:
        print(f"Appending to existing data file: {data_output_csv}")

    print("Starting download process...")

    # Iterate through each row in the input file
    for index, row in df.iterrows():
        proteome_id = str(row['Proteome Id']).strip()
        organism_id = str(row['Organism Id']).strip()
        expected_count = int(row['Protein count']) if pd.notnull(row['Protein count']) else 0
        
        # --- Step 3: Download protein sequences from UniProtKB in TSV format ---

        # Construct the query URL using both Proteome ID and Organism ID
        # We use 'proteome:ID AND organism_id:ID'
        # We use urllib.parse.quote to ensure the query string is URL-safe
        query_string = f"proteome:{proteome_id} AND organism_id:{organism_id}"
        encoded_query = urllib.parse.quote(query_string)
        # We fetch 'accession' (Entry) and 'sequence'
        query_url = f"https://rest.uniprot.org/uniprotkb/stream?query={encoded_query}&format=tsv&fields=accession,sequence"
        
        try:
            response = requests.get(query_url, stream=True)
            response.raise_for_status()
            
            # Read the TSV data into a pandas DataFrame
            # io.StringIO converts the string output to a file-like object for pandas
            # on_bad_lines='skip' helps avoid crashing on malformed lines
            downloaded_df = pd.read_csv(io.StringIO(response.text), sep='\t')
            
            # Calculate counts
            downloaded_count = len(downloaded_df)
            mismatch_count = abs(expected_count - downloaded_count)
            
            # --- Step 4: Verify counts ---
            status = "Match" if mismatch_count == 0 else "Mismatch"
            print(f"[{index+1}/{len(df)}] {proteome_id}: Exp={expected_count}, Got={downloaded_count} ({status})")

            # --- Step 5: Save the final data to a csv file ---
            # Add the identifier columns to the downloaded data
            downloaded_df.insert(0, 'Proteome Id', proteome_id)
            downloaded_df.insert(1, 'Organism Id', organism_id)
            
            # Rename columns to match our output format if necessary (UniProt gives 'Entry', 'Sequence')
            downloaded_df.columns = ['Proteome Id', 'Organism Id', 'Entry', 'Sequence']
            
            # Append to the main CSV file
            # mode='a' appends, header=False prevents writing the header again
            downloaded_df.to_csv(data_output_csv, mode='a', header=False, index=False)

            # Record Metadata
            metadata_records.append({
                'Proteome Id': proteome_id,
                'Organism Id': organism_id,
                'Expected Count': expected_count,
                'Downloaded Count': downloaded_count,
                'Mismatch Count': mismatch_count,
                'Status': status,
                'Error': 'None'
            })

        except Exception as e:
            print(f"[{index+1}/{len(df)}] {proteome_id}: Error - {e}")
            
            # Log the error in metadata
            metadata_records.append({
                'Proteome Id': proteome_id,
                'Organism Id': organism_id,
                'Expected Count': expected_count,
                'Downloaded Count': 0,
                'Mismatch Count': expected_count,
                'Status': 'Error',
                'Error': str(e)
            })
        
        # Sleep briefly to be polite to the UniProt API
        time.sleep(0.5)

    # --- Generate Metadata Report ---
    metadata_df = pd.DataFrame(metadata_records)
    metadata_df.to_csv(metadata_output_csv, index=False)
    
    print("\nProcessing Complete!")
    print(f"Sequence Data Saved to: {data_output_csv}")
    print(f"Metadata Report Saved to: {metadata_output_csv}")

# ==========================================
# Execution Configuration
# ==========================================

# File paths
input_file = 'processed_proteomes_data.csv'       # The file from Step 1
data_file = 'halobacteriales_data.csv'            # The final output with sequences
metadata_file = 'halobacteriales_metadata.csv'    # The report file

# Run the function
if __name__ == "__main__":
    download_and_verify_proteomes(input_file, data_file, metadata_file)

Reading input file: processed_proteomes_data.csv...
Appending to existing data file: halobacteriales_data.csv
Starting download process...
[1/523] UP000037729: Exp=3788, Got=3788 (Match)
[2/523] UP000037747: Exp=3245, Got=3245 (Match)
[3/523] UP000066737: Exp=3340, Got=3340 (Match)
[4/523] UP000198775: Exp=5137, Got=5137 (Match)
[5/523] UP000198848: Exp=4046, Got=4046 (Match)
[6/523] UP000198888: Exp=3425, Got=3425 (Match)
[7/523] UP000198902: Exp=3877, Got=3877 (Match)
[8/523] UP000198932: Exp=2994, Got=2994 (Match)
[9/523] UP000199062: Exp=4091, Got=4091 (Match)
[10/523] UP000199076: Exp=4204, Got=4204 (Match)
[11/523] UP000199199: Exp=4009, Got=4009 (Match)
[12/523] UP000199370: Exp=3852, Got=3852 (Match)
[13/523] UP000216308: Exp=3137, Got=3137 (Match)
[14/523] UP000219453: Exp=3284, Got=3284 (Match)
[15/523] UP000236755: Exp=2791, Got=2791 (Match)
[16/523] UP000242917: Exp=3614, Got=3614 (Match)
[17/523] UP000243250: Exp=3607, Got=3607 (Match)
[18/523] UP000250088: Exp=3626, Got=3

In [2]:
df = pd.read_csv("halobacteriales_data.csv")
len(df)

1217223